In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque

# Global variable for start button
button_clicked = False

def on_mouse_click(event, x, y, flags, param):
    global button_clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        button_x, button_y = 500, 500
        button_width, button_height = 200, 100
        if (button_x <= x <= button_x + button_width and 
            button_y <= y <= button_y + button_height):
            button_clicked = True

def show_start_screen():
    """Display the start screen with a clickable button"""
    global button_clicked
    button_clicked = False
    
    # Create start screen
    start_screen = np.ones((720, 1280, 3), dtype=np.uint8) * 255
    
    # Add title
    cv2.putText(start_screen, "Air Canvas", (400, 200), 
                cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 5)
    
    # Add instructions
    instructions = [
        "Instructions:",
        "1. Use index finger to draw",
        "2. Two fingers up to move without drawing",
        "3. All fingers up to erase",
        "",
        "Click START to begin"
    ]
    
    for i, text in enumerate(instructions):
        cv2.putText(start_screen, text, (100, 300 + i*40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    
    # Create start button
    button_x, button_y = 500, 500
    button_width, button_height = 200, 100
    cv2.rectangle(start_screen, (button_x, button_y), 
                  (button_x + button_width, button_y + button_height), 
                  (200, 200, 200), -1)
    cv2.putText(start_screen, "START ", (button_x + 50, button_y + 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 3)
    
    cv2.namedWindow("Air Canvas - Start")
    cv2.setMouseCallback("Air Canvas - Start", on_mouse_click)
    
    while True:
        cv2.imshow("Air Canvas - Start", start_screen)
        key = cv2.waitKey(1)
        
        if button_clicked:
            cv2.destroyAllWindows()
            return True
            
        if key == ord('q') or cv2.getWindowProperty("Air Canvas - Start", cv2.WND_PROP_VISIBLE) < 1:
            cv2.destroyAllWindows()
            return False

def run_air_canvas():
    """Main Air Canvas application with hand gesture controls"""
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()

    # Initialize canvas
    canvas = np.ones((720, 1280, 3), dtype=np.uint8) * 255
    prev_x, prev_y = 0, 0
    drawing_color = (0, 255, 0)  # Green by default
    thickness = 5
    eraser_mode = False
    undo_stack = deque(maxlen=100)

    # Button setup
    button_width, button_height = 100, 60
    button_margin = 10
    buttons = {
        "Clear": (button_margin, button_margin),
        "Undo": (button_margin * 2 + button_width, button_margin),
        "Erase": (button_margin * 3 + button_width * 2, button_margin),
        "Red": (button_margin * 4 + button_width * 3, button_margin),
        "Green": (button_margin * 5 + button_width * 4, button_margin),
        "Blue": (button_margin * 6 + button_width * 5, button_margin),
        "Quit": (button_margin * 7 + button_width * 6, button_margin),
    }

    def is_index_finger_up(landmarks):
        return (landmarks.landmark[8].y < landmarks.landmark[6].y and
                landmarks.landmark[12].y > landmarks.landmark[10].y and
                landmarks.landmark[16].y > landmarks.landmark[14].y and
                landmarks.landmark[20].y > landmarks.landmark[18].y)

    def is_two_fingers_up(landmarks):
        return (landmarks.landmark[8].y < landmarks.landmark[6].y and
                landmarks.landmark[12].y < landmarks.landmark[10].y and
                landmarks.landmark[16].y > landmarks.landmark[14].y and
                landmarks.landmark[20].y > landmarks.landmark[18].y)

    def is_all_fingers_up(landmarks):
        return (landmarks.landmark[4].y < landmarks.landmark[3].y and
                landmarks.landmark[8].y < landmarks.landmark[6].y and
                landmarks.landmark[12].y < landmarks.landmark[10].y and
                landmarks.landmark[16].y < landmarks.landmark[14].y and
                landmarks.landmark[20].y < landmarks.landmark[18].y)

    def draw_buttons(frame):
        for text, (x, y) in buttons.items():
            color = (200, 200, 200)
            cv2.rectangle(frame, (x, y), (x + button_width, y + button_height), color, -1)
            cv2.putText(frame, text, (x + 10, y + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    def is_button_hovered(x, y):
        for (btn_x, btn_y) in buttons.values():
            if btn_x <= x <= btn_x + button_width and btn_y <= y <= btn_y + button_height:
                return True
        return False

    def handle_button_click(x, y):
        nonlocal drawing_color, eraser_mode, canvas, undo_stack
        for text, (btn_x, btn_y) in buttons.items():
            if btn_x <= x <= btn_x + button_width and btn_y <= y <= btn_y + button_height:
                if text == "Clear":
                    canvas = np.ones((720, 1280, 3), dtype=np.uint8) * 255
                    undo_stack.clear()
                elif text == "Undo" and undo_stack:
                    canvas = undo_stack.pop()
                elif text == "Erase":
                    eraser_mode = not eraser_mode
                    drawing_color = (255, 255, 255) if eraser_mode else (0, 255, 0)
                elif text == "Red":
                    drawing_color = (0, 0, 255)
                    eraser_mode = False
                elif text == "Green":
                    drawing_color = (0, 255, 0)
                    eraser_mode = False
                elif text == "Blue":
                    drawing_color = (255, 0, 0)
                    eraser_mode = False
                elif text == "Quit":
                    return True
        return False

    cv2.namedWindow("Canvas", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("Canvas", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        canvas_with_pointer = canvas.copy()

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_tip.x * frame.shape[1]), int(index_tip.y * frame.shape[0])

                cv2.circle(frame, (x, y), 10, (0, 255, 0), 2)
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
                cv2.circle(canvas_with_pointer, (x, y), 10, (200, 200, 200), 2)
                cv2.circle(canvas_with_pointer, (x, y), 3, (0, 0, 255), -1)

                if is_index_finger_up(hand_landmarks):
                    if handle_button_click(x, y):
                        break

                if is_index_finger_up(hand_landmarks) and not is_button_hovered(x, y):
                    if prev_x != 0 and prev_y != 0:
                        undo_stack.append(canvas.copy())
                        cv2.line(canvas, (prev_x, prev_y), (x, y), drawing_color, thickness)
                    prev_x, prev_y = x, y

                elif is_two_fingers_up(hand_landmarks):
                    prev_x, prev_y = x, y

                elif is_all_fingers_up(hand_landmarks) and not is_button_hovered(x, y):
                    if prev_x != 0 and prev_y != 0:
                        undo_stack.append(canvas.copy())
                        cv2.line(canvas, (prev_x, prev_y), (x, y), (255, 255, 255), thickness*2)
                    prev_x, prev_y = x, y

                else:
                    prev_x, prev_y = 0, 0

                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        draw_buttons(frame)
        cv2.imshow("Canvas", canvas_with_pointer)
        cv2.imshow("AirCanvas", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the application
if show_start_screen():
    run_air_canvas()